Displaying full slicer Viewer in separate tab

In [33]:
import JupyterNotebooksLib as slicernb
import slicer
slicer.mrmlScene.Clear(False)
# Load a Slicer sample data set
import SampleData
import SimpleITK as sitk
import sitkUtils

# Set viewer size to 50% of screen size
slicernb.AppWindow.setWindowSize(scale=0.5)
# Hide patient information from slice view
slicernb.showSliceViewAnnotations(False)

#slicernb.showVolumeRendering(volume, show=True)
app = slicernb.AppWindow()
app
app.setContents("full")
slicer.util.selectModule("Data")
from ipywidgets import HTML
HTML(f"""<a href="{slicernb.AppWindow.defaultDesktopUrl()}" target="_blank">
<b>Click here</b> to open application window in a new browser tab.</a>""")

HTML(value='<a href="/desktop/" target="_blank">\n<b>Click here</b> to open application window in a new browse…

In [20]:
def displayy(path,Name):    
    image=sitk.ReadImage(path)
    outputVolumeNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLScalarVolumeNode", Name)
    sitkUtils.PushVolumeToSlicer(image, outputVolumeNode)

In [35]:
import pandas as pd
import SimpleITK as sitk

df = pd.read_csv('/home/sliceruser/data/metadata/processedMetaData.csv')
row= df.iloc[19]
pathT2w= row['t2w']
pathHbv= row['hbv']
pathADC= row['adc']
patht2wLabel= row['reSampledPath']

registeredPath = '/home/sliceruser/data/exploration/result.0.mha'

def displayy(path,Name):    
    image=sitk.ReadImage(path)
    outputVolumeNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLScalarVolumeNode", Name)
    sitkUtils.PushVolumeToSlicer(image, outputVolumeNode)
    
displayy(pathT2w,'t2w')    
displayy(pathHbv,'origHbv')    
displayy(registeredPath,'registeredHbv')    


In [18]:
# path ='/home/sliceruser/data/orig/10001/10001_1000001_hbv_bf_corr.mha'
testImagePath = '/home/sliceruser/data/exploration/resampleddB.nii.gz'
origPath ='/home/sliceruser/data/orig/10019/10019_1000019_t2w.mha'
testImagePathB = '/home/sliceruser/data/exploration/resampledd_itk.nii.gz'

displayy(testImagePath,"my_resampled")
displayy(testImagePathB,"itk_resampled")
displayy(origPath,"original")

In [7]:
a='/home/sliceruser/data/Unires/ur_10019_1000019_adc.nii.gz'
b='/home/sliceruser/data/Unires/ur_10019_1000019_cor.nii.gz'
c='/home/sliceruser/data/Unires/ur_10019_1000019_hbv.nii.gz'
d='/home/sliceruser/data/Unires/ur_10019_1000019_sag.nii.gz'
e='/home/sliceruser/data/Unires/ur_10019_1000019_t2w.nii.gz'


def displayy(path,Name):    
    image=sitk.ReadImage(path)
    outputVolumeNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLScalarVolumeNode", Name)
    sitkUtils.PushVolumeToSlicer(image, outputVolumeNode)
    
displayy(a,'adc')
displayy(b,'cor')
displayy(c,'hbv')
displayy(d,'sag')
displayy(e,'t2w')

    

In [22]:
import pandas as pd
import SimpleITK as sitk

df = pd.read_csv('/home/sliceruser/data/metadata/processedMetaData.csv')
row= df.iloc[19]
pathT2w= row['t2w']
pathHbv= row['hbv']
pathADC= row['adc']= row['adc']
patht2wLabel= row['reSampledPath']

def displayy(path,Name):    
    image=sitk.ReadImage(path)
    outputVolumeNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLScalarVolumeNode", Name)
    sitkUtils.PushVolumeToSlicer(image, outputVolumeNode)
    
displayy(pathT2w,"t2w")    
displayy(pathHbv,"hbv")    
displayy(pathADC,"adc")    
displayy(patht2wLabel,"label")    


/tmp/xpython_1235/2858738171.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pathADC= row['adc']= row['adc']


In [26]:
image=sitk.ReadImage(registeredPath)
print(image.GetSize())
image=sitk.ReadImage(pathT2w)
print(image.GetSize())

(384, 384, 19)
(384, 384, 19)


In [10]:
node = slicer.cli.runSync(slicer.modules.slicerelastixcli)
print(node)

<class 'AttributeError'>: module 'modules' has no attribute 'slicerelastixcli'

In [9]:
slicer.modules

<module 'modules' from '/home/sliceruser/Slicer/bin/Python/slicer/__init__.py'>

In [18]:
import pandas as pd
import SimpleITK as sitk

df = pd.read_csv('/home/sliceruser/data/metadata/processedMetaData.csv')
row= df.iloc[19]
pathT2w= row['t2w']
pathHbv= row['hbv']
pathADC= row['adc']
patht2wLabel= row['reSampledPath']
outputPath= '/home/sliceruser/data/exploration/outt.mha'
#as in https://discourse.slicer.org/t/accessing-slicerelastix-from-within-python-module/9410/5
# import Elastix
# elastixLogic = Elastix.ElastixLogic()
# parameterFilenames = elastixLogic.getRegistrationPresets()[0][Elastix.RegistrationPresets_ParameterFilenames]
# fixedVolume=pathT2w
# movingVolume=pathHbv
# outputVol = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLScalarVolumeNode", " output elastix ")
# outputTransform = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLTransformNode")

# elastixLogic.registerVolumes(
# fixedVolume, movingVolume,
# outputVolumeNode = outputVol,
# parameterFilenames = parameterFilenames,
# outputTransformNode = outputTransform
#)
print(pathT2w)
print(pathHbv)
print(outputPath)

/home/sliceruser/data/orig/10019/10019_1000019_t2w.mha
/home/sliceruser/data/orig/10019/10019_1000019_hbv.mha
/home/sliceruser/data/exploration/resampleddB.nii.gz
